# Performance Comparison of Agentic-RAG vs RAG

Retrieval-Augmented-Generation (RAG) involves using a large language model (LLM) to answer user queries based on information retrieved from a knowledge base. This approach offers several advantages over using a vanilla or fine-tuned LLM. It allows grounding answers in factual information, reducing confabulations, providing domain-specific knowledge, and enabling fine-grained control over access to information from the knowledge base.

However, RAG has some limitations, particularly:

1. It performs only one retrieval step: if the initial retrieval results are poor, the generated answer will also be poor.
2. Semantic similarity is computed with the user query as a reference, which may be suboptimal. For example, user queries are often   questions, while documents containing the true answers are in affirmative form. This discrepancy can lead to relevant information being missed.

These issues can be mitigated by creating a RAG agent, which is essentially an agent equipped with a retriever tool. This agent can:

- Formulate the query itself.
- Critique and re-retrieve information if necessary.

By doing so, the agent can recover some advanced RAG techniques. Instead of using the user query directly in semantic search, the agent formulates a reference sentence that is closer to the targeted documents, similar to the HyDE approach. Additionally, the agent can generate snippets and re-retrieve information as needed, akin to the Self-Query approach.

In [9]:
%pip install pandas langchain langchain-community sentence-transformers faiss-cpu smolagents --upgrade -q

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install ipywidgets -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
from huggingface_hub import notebook_login

notebook_login()

# Agentic RAG

In [7]:
%pip install datasets -q

Note: you may need to restart the kernel to use updated packages.


first load a knowledge base on which we want to perform RAG: this dataset is a compilation of the documentation pages for many huggingface packages, stored as markdown.

In [6]:
import datasets

knowledge_bas = datasets.load_dataset("m-ric/huggingface_doc", split="train")

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

d:\AI-Agents\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hrith\.cache\huggingface\hub\datasets--m-ric--huggingface_doc. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

Now prepare the knowledge base by processing the dataset and storing it into a vector database to be used by the retriever.

We use LangChain for its excellent vector database utilities. For the embedding model, we use thenlper/gte-small.

In [12]:
from tqdm import tqdm
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

source_doc = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_bas
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

#split dics and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_doc):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[new_doc.page_content] = True
            docs_processed.append(new_doc)

print("Embedding documents...")

embedding_mode = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_mode,
    distance_strategy=DistanceStrategy.COSINE,
)

Splitting documents...


100%|██████████| 2647/2647 [00:36<00:00, 71.86it/s] 
C:\Users\hrith\AppData\Local\Temp\ipykernel_23996\1882312669.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_mode = HuggingFaceEmbeddings(model_name="thenlper/gte-small")


Embedding documents...


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding documents Take around 17 mins on CPU (Windows PC)

Database is ready. Now Building Agentic RAG. We only need a RetrieverTool that our agent can leverage to retrieve information from the knowledge base.

In [13]:
from smolagents import Tool
from langchain_core.vectorstores import VectorStore


class RetrieverTool(Tool):
    name = "retriever"
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.vectordb.similarity_search(
            query,
            k=7,
        )

        return "\nRetrieved documents:\n" + "".join(
            [
                f"===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

Now create an agent that leverages this tool!

In [15]:
from smolagents import LiteLLMModel, ToolCallingAgent

model = LiteLLMModel(
    model_id="ollama/qwen2.5:7b",
    api_base="http://127.0.0.1:11434",
    num_ctx=8192,
)

retriever_tool = RetrieverTool(vectordb)
agent = ToolCallingAgent(tools=[retriever_tool], model=model)

In [16]:
agent_output = agent.run("How can I push a model to the Hub?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How can I push a model to the Hub?                                                                              │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5:7b ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'retriever' with arguments: {'query': 'how to push a model to Hugging Face Hub'}                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved documents:
===== Document 0 =====
Finally, if you want, you can push your model up to the hub. Here, we'll push it up if you specified 
`push_to_hub=True` in the training configuration. Note that in order to push to hub, you'll have to have git-lfs 
installed and be logged into your Hugging Face account (which can be done via `huggingface-cli login`).

```python
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "image-classification",
    "dataset": 'beans',
    "tags": |'image-classification'],
}===== Document 1 =====
### 6. Which of the following are valid ways of loading a Hugging Face model from Hub or Spaces?===== Document 2 
=====
Hub methods

Methods for using the Hugging Face Hub:

## Push to hub 

||autodoc]] evaluate.push_to_hub===== Document 3 =====
he Push to Hub API. Let's have a look at the push_to_hub API. You will need to be logged in with your Hugging Face 
account, which you can do by executing this first cell or typing huggingface-cli login in a terminal. Just enter 
your username and password and click login, which will store an authentication token in the cache of the machine 
you're using. Now, let's launch the fine-tuning of a BERT model on the GLUE COLA dataset. We won't go over the 
fine-tuning code because you can find it in any Transformers tutorial, or by looking at the videos linked below. 
What interests us here, is how we can leverage the Model Hub during training===== Document 4 =====
# Pushing all things after training
trainer.push_to_hub()
```

There is much more you can do, so we suggest to review the |Share a 
model](https://huggingface.co/docs/transformers/model_sharing) guide.

## Additional resources

* Transformers |library](https://github.com/huggingface/transformers).
* Transformers |docs](https://huggingface.co/docs/transformers/index).
* Share a model |guide](https://huggingface.co/docs/transformers/model_sharing).===== Document 5 =====
The `push_to_hub()` method is backed by the |`huggingface_hub`](https://github.com/huggingface/huggingface_hub) 
Python package, which offers a direct API to the Hugging Face Hub. It's integrated within 🤗 Transformers and 
several other machine learning libraries, like |`allenlp`](https://github.com/allenai/allennlp). Although we focus 
on the 🤗 Transformers integration in this chapter, integrating it into your own code or library is simple.

Jump to the last section to see how to upload files to your newly created repository!

## Using the `huggingface_hub` Python library||using-the-huggingfacehub-python-library]]===== Document 6 =====
Once you have a trained topic model, you can push it to the Hugging Face Hub in one line. Pushing your model to the
Hub will automatically create an initial model card for your model, including an overview of the topics created. 
Below you can see an example of the topics resulting from a |model trained on ArXiv 
data](https://huggingface.co/MaartenGr/BERTopic_ArXiv).

[Step 1: Duration 14.92 seconds| Input tokens: 1,168 | Output tokens: 31]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'To push a model to the Hugging Face Hub, you need to   │
│ be logged into your Hugging Face account. You can do this via `huggingface-cli login`. If you specified         │
│ `push_to_hub=True` in your training configuration, the model will be automatically pushed after training.       │
│ Otherwise, you can use the `trainer.push_to_hub()` method during or after training. For more detailed           │
│ instructions and additional resources, refer to the Share a model guide available at                            │
│ https://huggingface.co/docs/transformers/model_sharing.'}                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: To push a model to the Hugging Face Hub, you need to be logged into your Hugging Face account. You 
can do this via `huggingface-cli login`. If you specified `push_to_hub=True` in your training configuration, the 
model will be automatically pushed after training. Otherwise, you can use the `trainer.push_to_hub()` method during
or after training. For more detailed instructions and additional resources, refer to the Share a model guide 
available at https://huggingface.co/docs/transformers/model_sharing.

[Step 2: Duration 11.19 seconds| Input tokens: 3,160 | Output tokens: 161]

Final output:
To push a model to the Hugging Face Hub, you need to be logged into your Hugging Face account. You can do this via `huggingface-cli login`. If you specified `push_to_hub=True` in your training configuration, the model will be automatically pushed after training. Otherwise, you can use the `trainer.push_to_hub()` method during or after training. For more detailed instructions and additional resources, refer to the Share a model guide available at https://huggingface.co/docs/transformers/model_sharing.


# Standard RAG

In [17]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

d:\AI-Agents\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hrith\.cache\huggingface\hub\datasets--m-ric--huggingface_doc_qa_eval. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65 [00:00<?, ? examples/s]

In [18]:
outputs_agentic_rag = []

for example in tqdm(eval_dataset):
    question = example["question"]

    enhanced_question = f"""Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".

Question:
{question}"""
    answer = agent.run(enhanced_question)
    print("=======================================================")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f'True answer: {example["answer"]}')

    results_agentic = {
        "question": question,
        "true_answer": example["answer"],
        "source_doc": example["source_doc"],
        "generated_answer": answer,
    }
    outputs_agentic_rag.append(results_agentic)

  0%|          | 0/65 [00:00<?, ?it/s]

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Using the information contained in your knowledge base, which you can access with the 'retriever' tool,         │
│ give a comprehensive answer to the question below.                                                              │
│ Respond only to the question asked, response should be concise and relevant to the question.                    │
│ If you cannot find information, do not give up and try calling your retriever again with different arguments!   │
│ Make sure to have covered the question completely by calling the retriever tool several times with semantically │
│ different queries.                                                                                              │
│ Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model    │
│ from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".                               │
│                                                                                                                 │
│ Question:                                                                                                       │
│ What architecture is the `tokenizers-linux-x64-musl` binary designed for?                                       │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5:7b ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'retriever' with arguments: {'query': 'architecture of tokenizers-linux-x64-musl'}                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved documents:
===== Document 0 =====
`tokenizers-linux-x64-musl`

This is the **x86_64-unknown-linux-musl** binary for `tokenizers`===== Document 1 =====
`tokenizers-linux-arm64-musl`

This is the **aarch64-unknown-linux-musl** binary for `tokenizers`===== Document 2 =====
`tokenizers-linux-x64-gnu`

This is the **x86_64-unknown-linux-gnu** binary for `tokenizers`===== Document 3 =====
`tokenizers-linux-arm64-gnu`

This is the **aarch64-unknown-linux-gnu** binary for `tokenizers`===== Document 4 =====
`tokenizers-freebsd-x64`

This is the **x86_64-unknown-freebsd** binary for `tokenizers`===== Document 5 =====
# Tokenizers

Provides an implementation of today's most used tokenizers, with a focus on performance and
versatility.

Bindings over the |Rust](https://github.com/huggingface/tokenizers/tree/master/tokenizers) implementation.
If you are interested in the High-level design, you can go check it there.

Otherwise, let's dive in!

## Main features:===== Document 6 =====
`tokenizers-win32-x64-msvc`

This is the **x86_64-pc-windows-msvc** binary for `tokenizers`

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'To install the latest version of Optimum with OpenVINO │
│ support, you should use the following command: pip install --upgrade-strategy eager optimum[openvino]'}         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: To install the latest version of Optimum with OpenVINO support, you should use the following command:
pip install --upgrade-strategy eager optimum[openvino]

[Step 2: Duration 4.38 seconds| Input tokens: 3,544 | Output tokens: 80]

100%|██████████| 65/65 [11:56<00:00, 11.02s/it]

Question: What is the command to install the latest version of Optimum with OpenVINO support?

Answer: To install the latest version of Optimum with OpenVINO support, you should use the following command: pip install --upgrade-strategy eager optimum[openvino]
True answer: pip install --upgrade-strategy eager optimum["openvino"]


In [ ]:
from tqdm import tqdm
from litellm import completion  # Import litellm directly

outputs_standard_rag = []

for example in tqdm(eval_dataset):
    question = example["question"]
    context = retriever_tool(question)

    prompt = f"""Given the question and supporting documents below, give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If you cannot find information, do not give up and try calling your retriever again with different arguments!

Question:
{question}

{context}
"""
    # Generate the answer using litellm.completion
    response = completion(
        model="ollama/qwen2.5:7b",  # Match your model
        messages=[{"content": prompt, "role": "user"}],  # Format as a message
        api_base="http://127.0.0.1:11434",
    )
    answer = response.choices[0].message.content  # Extract the text from the response

    print("=======================================================")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f'True answer: {example["answer"]}')

    results_agentic = {
        "question": question,
        "true_answer": example["answer"],
        "source_doc": example["source_doc"],
        "generated_answer": answer,
    }
    outputs_standard_rag.append(results_agentic)

  2%|▏         | 1/65 [00:10<11:01, 10.33s/it]

Question: What architecture is the `tokenizers-linux-x64-musl` binary designed for?

Answer: The `tokenizers-linux-x64-musl` binary is designed for the **x86_64-unknown-linux-musl** architecture.
True answer: x86_64-unknown-linux-musl


  3%|▎         | 2/65 [00:14<07:11,  6.85s/it]

Question: What is the purpose of the BLIP-Diffusion model?

Answer: The purpose of the BLIP-Diffusion model is to enable zero-shot subject-driven generation and control-guided zero-shot generation. This model was proposed in the paper [BLIP-Diffusion: Pre-trained Subject Representation for Controllable Text-to-Image Generation and Editing](https://arxiv.org/abs/2305.14720).
True answer: The BLIP-Diffusion model is designed for controllable text-to-image generation and editing.


  5%|▍         | 3/65 [00:24<08:18,  8.05s/it]

...

Question: What is the name of the open-source library created by Hugging Face to simplify Transformer acceleration?

Answer: The name of the open-source library created by Hugging Face to simplify Transformer acceleration is **Optimum**. This information can be found in Document 0.
True answer: Optimum


 98%|█████████▊| 64/65 [05:15<00:03,  3.52s/it]

Question: What parameter is used to ensure that elements in a row have the same height in Gradio?

Answer: The parameter used to ensure that elements in a row have the same height in Gradio is `equal_height`. This can be applied by passing it as an argument to the `.style()` method of `gr.Row()`. 

Source Document: Document 1
True answer: equal_height


100%|██████████| 65/65 [05:18<00:00,  4.90s/it]

Question: What is the command to install the latest version of Optimum with OpenVINO support?

Answer: The command to install the latest version of Optimum with OpenVINO support is:

```bash
pip install --upgrade-strategy eager optimum["openvino"]
```

This can be found in Document 1.
True answer: pip install --upgrade-strategy eager optimum["openvino"]


The evaluation prompt follows some of the best principles shown in our llm_judge cookbook: it follows a small integer Likert scale, has clear criteria, and a description for each score.

In [33]:
EVALUATION_PROMPT = """You are a fair evaluator language model.

You will be given an instruction, a response to evaluate, a reference answer that gets a score of 3, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 3. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 3}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.
5. Do not score conciseness: a correct answer that covers the question should receive max score, even if it contains additional useless information.

The instruction to evaluate:
{instruction}

Response to evaluate:
{response}

Reference Answer (Score 3):
{reference_answer}

Score Rubrics:
[Is the response complete, accurate, and factual based on the reference answer?]
Score 1: The response is completely incomplete, inaccurate, and/or not factual.
Score 2: The response is somewhat complete, accurate, and/or factual.
Score 3: The response is completely complete, accurate, and/or factual.

Feedback:"""

In [48]:
class EvaluationClient:
    def __init__(self, model="ollama/llama3.1:8b", api_base="http://127.0.0.1:11434"):
        self.model = model
        self.api_base = api_base

    def __call__(self, prompt):
        response = completion(
            model=self.model,
            messages=[{"content": prompt, "role": "user"}],
            api_base=self.api_base,
            max_tokens=1000  # Add this to limit output length
        )
        return response.choices[0].message.content

# Instantiate with the updated class
evaluation_client = EvaluationClient("ollama/llama3.1:8b")

In [49]:
import pandas as pd

results = {}
for system_type, outputs in [
    ("agentic", outputs_agentic_rag),
    ("standard", outputs_standard_rag),
]:
    for experiment in tqdm(outputs):
        eval_prompt = EVALUATION_PROMPT.format(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        messages = [
            {"role": "system", "content": "You are a fair evaluator language model."},
            {"role": "user", "content": eval_prompt},
        ]

        eval_result = evaluation_client(eval_prompt)
        try:
            feedback, score = [item.strip() for item in eval_result.split("[RESULT]")]
            experiment["eval_score_LLM_judge"] = score
            experiment["eval_feedback_LLM_judge"] = feedback
        except:
            print(f"Parsing failed - output was: {eval_result}")

    results[system_type] = pd.DataFrame.from_dict(outputs)
    results[system_type] = results[system_type].loc[~results[system_type]["generated_answer"].str.contains("Error")]

  5%|▍         | 3/65 [00:17<07:08,  6.90s/it]

Parsing failed - output was: Feedback: Step 1 of the response accurately describes associating an email with a Hugging Face Hub account to claim authorship. [RESULT] 2
 
Feedback: However, step 3 of the response inaccurately states that clicking on one's name and then "claim authorship" redirects to paper settings where one can confirm the request. The correct procedure is simply clicking "claim authorship". [RESULT] 1
 
Feedback: The entire process outlined in the response accurately describes how a user claims authorship of a paper on the Hugging Face Hub, including waiting for admin team validation after submitting the claim and manually linking papers to an account if necessary. This matches the reference answer exactly. [RESULT] 3


100%|██████████| 65/65 [04:50<00:00,  4.47s/it]


In [50]:
DEFAULT_SCORE = 2 # Give average score whenever scoring fails
def fill_score(x):
    try:
        return int(x)
    except:
        return DEFAULT_SCORE

for system_type, outputs in [
    ("agentic", outputs_agentic_rag),
    ("standard", outputs_standard_rag),
]:

    results[system_type]["eval_score_LLM_judge_int"] = (
        results[system_type]["eval_score_LLM_judge"].fillna(DEFAULT_SCORE).apply(fill_score)
    )
    results[system_type]["eval_score_LLM_judge_int"] = (results[system_type]["eval_score_LLM_judge_int"] - 1) / 2

    print(
        f"Average score for {system_type} RAG: {results[system_type]['eval_score_LLM_judge_int'].mean()*100:.1f}%"
    )

Average score for agentic RAG: 79.2%
Average score for standard RAG: 88.5%
